In [68]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO
import sqlite3
import re
import json

In [54]:

def modifyRevenue(rev):
    while (len(rev) > 0 and rev[0] != '$'):
        rev = rev[1:]
    return rev

BASE_URL = 'https://en.wikipedia.org'
HEADERS = {'User-Agent': 'Mozilla/5.0'}
table_class = 'wikitable sortable plainrowheaders sticky-header col4right col5center col6center'

response = requests.get(BASE_URL + '/wiki/List_of_highest-grossing_films', headers=HEADERS)
soup = BeautifulSoup(response.text, 'html.parser')

film_table = soup.find('table', attrs={'class': table_class, 'style': 'margin:auto; margin:auto;'})

df = pd.read_html(StringIO(str(film_table)))[0]
df.drop(columns=['Ref', 'Peak'], inplace=True)

df['Worldwide gross'] = df['Worldwide gross'].apply(modifyRevenue)

countries = []
directors = []

for row in film_table.find_all('tr')[1:]:  
    url = row.select_one('a').get('href')
    #print(url)
    response = requests.get(BASE_URL + url, headers=HEADERS)
    soup = BeautifulSoup(response.text, 'html.parser')
    infobox = soup.find('table', {'class': 'infobox vevent'})

    country_of_origin = []
    curr_directors = []
    
    if not infobox: 
        countries.append('N/A')
        directors.append('N/A')
        continue

    for row in infobox.find_all('tr'):
        header = row.find('th')
        if header:
            header_text = header.get_text().strip()
            if header_text == 'Country' or header_text == 'Countries':
                # Find the next sibling <td> element
                country_cell = row.find('td')
                if country_cell:
                    country_of_origin = [text.strip() for text in country_cell.stripped_strings]
                    # Remove unwanted characters (e.g., [2]) using regex
                    country_of_origin = [re.sub(r'[^a-zA-Z\s]', '', country) for country in country_of_origin]
                    # Remove empty strings
                    country_of_origin = [country for country in country_of_origin if country.strip()]
            elif header_text == 'Directed by':
                director_cell = row.find('td')
                if director_cell:
                    # Extract all text, including nested <a> and <br> tags
                    curr_directors = [text.strip() for text in director_cell.stripped_strings]
                    # Remove unwanted characters (e.g., [2]) using regex
                    curr_directors = [re.sub(r'[^a-zA-Z\s]', '', director) for director in curr_directors]
                    # Remove empty strings
                    curr_directors = [director for director in curr_directors if director.strip()]

    #print(country_of_origin, curr_directors)
    countries.append(', '.join(country_of_origin))
    directors.append(', '.join(curr_directors))


df['country'] = countries
df['director'] = directors


In [ ]:
# assigning missing values manually
df.loc[32, 'director'] = 'Peter Jackson'
df.loc[32, 'country'] = 'New Zealand, Germany, United States'

df.loc[33, 'director'] = 'Jon Watts'
df.loc[33, 'country'] = 'United States'

df.loc[34, 'director'] = 'Anna Boden, Ryan Fleck'
df.loc[34, 'country'] = 'United States'

df.loc[41, 'director'] = 'Josh Cooley'
df.loc[41, 'country'] = 'United States'

df

In [ ]:
#Creating DB
conn = sqlite3.connect('films.db')

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Create table as per requirement
cursor.execute('DROP TABLE films')
sql = '''
    CREATE TABLE IF NOT EXISTS films (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT NOT NULL,
    release_year INTEGER,
    director TEXT,
    box_office TEXT,
    country TEXT
)'''

cursor.execute(sql)
conn.commit()

In [71]:
# Insert data into the table
for index, row in df.iterrows():
    cursor.execute('''
        INSERT INTO films (title, release_year, director, box_office, country)
        VALUES (?, ?, ?, ?, ?)
    ''', (row['Title'], row['Year'], row['director'], row['Worldwide gross'], row['country']))

# Commit the changes
conn.commit()

# Retrieve and display all rows from the films table
cursor.execute("SELECT * FROM films")
rows = cursor.fetchall()

# Convert rows to a list of dictionaries
films_list = []
for row in rows:
    film_dict = {
        'id': row[0],
        'title': row[1],
        'release_year': row[2],
        'director': row[3],
        'box_office': row[4],
        'country': row[5]
    }
    films_list.append(film_dict)

# Write the data to a JSON file
with open('films.json', 'w') as json_file:
    json.dump(films_list, json_file, indent=4)

# Close the connection
conn.close()

